# Imports and loads

In [7]:
import pickle
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer



In [5]:
def save_model(model,name):
    file_ext= '.sav'
    path = 'models/'
    pickle.dump(model, open(path+name+file_ext, 'wb'))
    
def predict_one(string, model_name, vectorizor_name):
    
    path = 'models/'
        
    tfid = pickle.load(open(path+vectorizor_name, 'rb'))
    tfidfed = tfid.transform([y_string])

    model = pickle.load(open(path+model_name, 'rb'))
    return model.predict(tfidfed)

def predict_many(review_list, model_name, vectorizor_name):
    path = 'models/'
        
    tfid = pickle.load(open(path+vectorizor_name, 'rb'))
    tfidfed = tfid.transform(review_list)

    model = pickle.load(open(path+model_name, 'rb'))
    return model.predict(tfidfed)

In [9]:
arr  = np.load('data/english_arr.npy', allow_pickle = True)


# Vectorize and Stem, or: Pre-handling

In [29]:
lang = arr[:,3]
ratings = arr[:,0]


In [ ]:
tfid = TfidfVectorizer(stop_words ='english', lowercase = True, max_features = 5000)
tfidfed = tfid.fit_transform(lang)

X = tfidfed
y = ratings.reshape(-1,1)

In [ ]:
cluster1_model = KMeans(n_clusters = 6, n_jobs =-1)
cluster1_model.fit_transform(X)

In [10]:
tf

array([5, '1927303-mary', ('Paperback 9781435756342',),
       'Live love laugh a Lot is a special book.Interesting characters make you laugh- you must take a look.Voicing their opinions in humorous forms moving line by line.Entertaining to such an extent- you must take your time.Love the way Jacquelyn speaks through Sly Fox.Over and over you have to laugh as he comes out of his box.Very sly he tries to be from behind his cloth.Excitement on each page you must laugh a lot.Laughter rings out as Ingrid does her stuff.A woman who could finally say, “Enough is Enough”.Use me Lord Jacquelyn says on page fifty-eight.God’s blessings comes in many ways Jacquelyn points out to us.Honesty and humor you will find within these pages so much.A great read and must for every home.Live Love Laugh a lotOh how humorousThis book is written by Jacquelyn Sturge ',
       'en'], dtype=object)